In [3]:
from keras.utils import to_categorical 
from keras.preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np

In [4]:
TRAIN_DIR = '/Users/prag/emotion/images/train'
TEST_DIR = '/Users/prag/emotion/images/test'

In [14]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label, "- Label Found")
    return image_paths,labels

In [15]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)

happy - Label Found
sad - Label Found
fear - Label Found
surprise - Label Found
neutral - Label Found
angry - Label Found
disgust - Label Found


In [16]:
print(train)

                                                   image    label
0        /Users/prag/emotion/images/train/happy/3578.jpg    happy
1       /Users/prag/emotion/images/train/happy/16988.jpg    happy
2        /Users/prag/emotion/images/train/happy/2666.jpg    happy
3        /Users/prag/emotion/images/train/happy/5109.jpg    happy
4       /Users/prag/emotion/images/train/happy/11981.jpg    happy
...                                                  ...      ...
28816  /Users/prag/emotion/images/train/disgust/10112...  disgust
28817  /Users/prag/emotion/images/train/disgust/21668...  disgust
28818  /Users/prag/emotion/images/train/disgust/7049.jpg  disgust
28819  /Users/prag/emotion/images/train/disgust/9716.jpg  disgust
28820  /Users/prag/emotion/images/train/disgust/3561.jpg  disgust

[28821 rows x 2 columns]


In [17]:
test = pd.DataFrame()
test ['image'], test['label'] = createdataframe(TEST_DIR)

happy - Label Found
sad - Label Found
fear - Label Found
surprise - Label Found
neutral - Label Found
angry - Label Found
disgust - Label Found


In [18]:
print(test)

                                                  image    label
0       /Users/prag/emotion/images/test/happy/23933.jpg    happy
1       /Users/prag/emotion/images/test/happy/24906.jpg    happy
2       /Users/prag/emotion/images/test/happy/18033.jpg    happy
3       /Users/prag/emotion/images/test/happy/15271.jpg    happy
4       /Users/prag/emotion/images/test/happy/26888.jpg    happy
...                                                 ...      ...
7061  /Users/prag/emotion/images/test/disgust/20761.jpg  disgust
7062  /Users/prag/emotion/images/test/disgust/28710.jpg  disgust
7063  /Users/prag/emotion/images/test/disgust/23876.jpg  disgust
7064   /Users/prag/emotion/images/test/disgust/9460.jpg  disgust
7065  /Users/prag/emotion/images/test/disgust/35580.jpg  disgust

[7066 rows x 2 columns]


In [20]:
from tqdm.notebook import tqdm

In [25]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image,color_mode='grayscale')
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features),48,48,1)
    return features

In [26]:
train_features = extract_features(train['image'])

  0%|          | 0/28821 [00:00<?, ?it/s]

In [28]:
test_features = extract_features(test['image'])

  0%|          | 0/7066 [00:00<?, ?it/s]

In [29]:
x_train = train_features/255.0
x_test = test_features/255.0

In [31]:
from sklearn.preprocessing import LabelEncoder

In [32]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [33]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [34]:
y_train = to_categorical(y_train,num_classes=7)
y_test = to_categorical(y_test, num_classes= 7)


In [35]:
model = Sequential()
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
#connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(7, activation='softmax'))


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [37]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [38]:
model.fit(x = x_train, y= y_train, batch_size=128, epochs=100, validation_data=(x_test, y_test))

Epoch 1/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 528s 2s/step - accuracy: 0.2394 - loss: 1.8362 - val_accuracy: 0.2583 - val_loss: 1.8059
Epoch 2/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 893s 4s/step - accuracy: 0.2479 - loss: 1.8017 - val_accuracy: 0.3023 - val_loss: 1.6981
Epoch 3/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 796s 4s/step - accuracy: 0.3196 - loss: 1.6762 - val_accuracy: 0.4312 - val_loss: 1.4667
Epoch 4/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 503s 2s/step - accuracy: 0.4036 - loss: 1.5250 - val_accuracy: 0.4808 - val_loss: 1.3677
Epoch 5/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 752s 3s/step - accuracy: 0.4417 - loss: 1.4391 - val_accuracy: 0.5003 - val_loss: 1.3098
Epoch 6/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 503s 2s/step - accuracy: 0.4601 - loss: 1.3854 - val_accuracy: 0.5110 - val_loss: 1.2776
Epoch 7/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 547s 2s/step - accuracy: 0.4825 - loss: 1.3497 - val_accuracy: 0.5219 - val_loss: 1.2498
Epoch 8/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 532s 2s/step - accuracy: 0.4970 - loss: 1.3168 - 

In [39]:
model_json = model.to_json()
with open("emotiondetector.json",'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")

In [40]:
from keras.models import model_from_json

In [41]:
json_file = open("emotiondetector.json","r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("emotiondetector.h5")

In [42]:
label = ['angry','disgust','fear','happy','neutral','sad','surprise']

In [44]:
def ef(image):
    img = load_img(image, color_mode='grayscale')
    feature = np.array(img)
    feature = feature.reshape(1,48,48,1)
    return feature/255.0

In [53]:
image = '/Users/prag/emotion/images/train/fear/2.jpg'
print('This is an original image of an fear expression!')
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print('Model Predection is', pred_label)

This is an original image of an fear expression!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
Model Predection is fear
